In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/car-price-prediction/CarPrice_Assignment.csv
/kaggle/input/car-price-prediction/Data Dictionary - carprices.xlsx


In [2]:
df = pd.read_csv("/kaggle/input/car-price-prediction/CarPrice_Assignment.csv")
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


## Data Cleaning

In [3]:
### Checking for null data 

df.isnull().sum()
#No Null data, so there is no need for value imputation

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [4]:
#to check simple statistic 
print("the length of the data is : ",len(df))


df.describe()



the length of the data is :  205


,car_ID,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,103.000000,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329756,3.255415,10.142537,104.117073,5125.121951,25.219512,30.751220,13276.710571
std,59.322565,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.270844,0.313597,3.972040,39.544167,476.985643,6.542142,6.886443,7988.852332
min,1.000000,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,52.000000,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7788.000000
50%,103.000000,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10295.000000
75%,154.000000,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.580000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16503.000000
max,205.000000,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000,45400.000000


In [5]:
#Removing Outliers 
numerical_columns = df.select_dtypes(exclude=['object']).columns

def remove_outliers(df, cols):
    Q1 = df[cols].quantile(0.25)
    Q3 = df[cols].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5*IQR
    return df[~((df[cols] < lower_bound) | (df[cols] > upper_bound)).any(axis=1)]

df = remove_outliers(df, numerical_columns)


### Categorical Variable 

In [6]:
## Working with categorical columns 


## CarName ##
df.select_dtypes(include=["object"])
print(df["CarName"].value_counts())
print("\n ")
print (" We have 102 variation for 139 dataset. It seems that CarName format is Car_Brand *space* Car_type  We will be using Car Brand instead of car type to gave more generalized view")
df["CarBrand"] = df["CarName"].str.split(" ").str[0]

##droping the CarName Columns
df = df.drop(columns = ["CarName"],axis =1 )





toyota corolla         5
toyota corona          5
mazda 626              3
toyota mark ii         3
mazda glc              2
                      ..
maxda glc deluxe       1
maxda rx3              1
isuzu D-Max V-Cross    1
isuzu MU-X             1
volvo diesel           1
Name: CarName, Length: 102, dtype: int64

 
 We have 102 variation for 139 dataset. It seems that CarName format is Car_Brand *space* Car_type  We will be using Car Brand instead of car type to gave more generalized view


In [7]:
## fueltype
print (df["fueltype"].value_counts())
print(" \n")
print("It seems that the option always gas, and hence there is no other variation and other distinguishable categories, hence we will drop the columns")

df = df.drop(columns = ["fueltype"],axis = 1 )

gas    139
Name: fueltype, dtype: int64
 

It seems that the option always gas, and hence there is no other variation and other distinguishable categories, hence we will drop the columns


In [8]:
##aspiration
print (df["aspiration"].value_counts())

## we will use get_dummies for this 

std      126
turbo     13
Name: aspiration, dtype: int64


In [9]:
## door_number

door_number_replacement = {"two" : 2 ,
                           "four" : 4}

df = df.replace({"door_number":door_number_replacement})

print("we are replacing door because it can be represented as number /")

we are replacing door because it can be represented as number /


In [10]:
##carbody

df["carbody"].unique()
## we will use dummies for this 

array(['convertible', 'hatchback', 'sedan', 'wagon', 'hardtop'],
      dtype=object)

In [11]:
df["drivewheel"].unique()
## we will use dummies for this 

array(['rwd', 'fwd', '4wd'], dtype=object)

In [12]:
print(df["enginelocation"].unique())

df = df.drop(columns = ["enginelocation"], axis = 1 )


print(" \n")
print("We will drop this because no other variation and other distinguishable categories, hence we will drop the columns")


['front']
 

We will drop this because no other variation and other distinguishable categories, hence we will drop the columns


In [13]:
print(df["enginetype"].unique())
#we will use one_hot_encoding for this 


['dohc' 'ohcv' 'ohc' 'rotor' 'l']


In [14]:
print(df["cylindernumber"].unique())

cylindernumber_replacement = {"four" : 4,
                            "six" : 6,
                            "five": 5,
                            "two" : 2 }

df = df.replace({"cylindernumber" : cylindernumber_replacement})

print("\n")
print("We replace cylindernumber from string to integer")

['four' 'six' 'five' 'two']


We replace cylindernumber from string to integer


In [15]:
print(df["fuelsystem"].unique())

#we willuse dummies for this 

['mpfi' '2bbl' '1bbl' 'spfi' '4bbl' 'spdi']


In [16]:
#Final dataset 
df.head()

,car_ID,symboling,aspiration,doornumber,carbody,drivewheel,wheelbase,carlength,carwidth,carheight,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,CarBrand
0,1,3,std,two,convertible,rwd,88.6,168.8,64.1,48.8,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,2,3,std,two,convertible,rwd,88.6,168.8,64.1,48.8,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,3,1,std,two,hatchback,rwd,94.5,171.2,65.5,52.4,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,4,2,std,four,sedan,fwd,99.8,176.6,66.2,54.3,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,5,2,std,four,sedan,4wd,99.4,176.6,66.4,54.3,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi


In [17]:
##get_dummies for categorical variabel

#pd.get_dummies(df["aspiration"], drop_first = True, prefix = "aspiration")

categorical_columns = df.select_dtypes(include=["object"]).columns

for i in categorical_columns : 
    dummies_creator = pd.get_dummies(df[i],drop_first= True, prefix = i )
    df = df.join(dummies_creator)
    df=df.drop(columns=[i])
    
    



### Min_Max_Scaling

In [18]:

def build_min_max_scaler(df) :
  scaled_columns = pd.DataFrame()
  for i in df.columns:
      min = df[i].min()
      df[i] = df[i]-min
      max = df[i].max()
      scaled_columns[i] = pd.DataFrame( round(df[i]/max ,5) )
  return scaled_columns


df_new = build_min_max_scaler(df.drop("price",axis = 1 ))
df = df_new.join(df["price"])

# Building Model

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
import statsmodels.api as sm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [20]:
X = df.drop("price",axis = 1 )
y = df["price"]

In [21]:
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.25)

In [22]:
X = sm.add_constant(X_train)
model = sm.OLS(y_train,X_train)
results = model.fit()
# Print the summary of the regression results
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                  price   R-squared (uncentered):                   0.994
Model:                            OLS   Adj. R-squared (uncentered):              0.989
Method:                 Least Squares   F-statistic:                              178.8
Date:                Mon, 15 May 2023   Prob (F-statistic):                    1.03e-45
Time:                        15:41:15   Log-Likelihood:                         -856.88
No. Observations:                 104   AIC:                                      1816.
Df Residuals:                      53   BIC:                                      1951.
Df Model:                          51                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

In [23]:
y_pred = results.predict(X_test)
mse = mean_absolute_error(y_test, y_pred)
mpe = abs (np.mean((y_test - y_pred) / y_test)) * 100
print("Mean Squarred Error :" , mse)
print("Mean Percentage Error : ",mpe)

Mean Squarred Error : 1984.5374161328018
Mean Percentage Error :  7.260021315976027
